In [ ]:
import os
import sys

src_path = os.path.abspath(os.path.join(".."))
if src_path not in sys.path:
    sys.path.append(src_path)

from src.data import navigation_events

In [ ]:
raw_events_df = navigation_events.load_events_data(os.path.join(os.pardir, "data", "navigation_events.csv"))

# Slide 1

In [ ]:
raw_events_df.head()

# Slide 2

In [ ]:
import altair as alt
alt.data_transformers.enable('data_server')
alt.renderers.enable('png')

membership_plot = alt.Chart(raw_events_df).mark_bar().encode(
    y = "membership_role",
    x = "count()",
    color = "membership_role",
    tooltip = "membership_role"
).properties(
    height = 250,
    width = 400
)

In [ ]:
membership_plot